In [ ]:
#### ECHILD Course Day 2: Developing e-Cohorts #### 

#### Part 1 ####

# pandas is a very helpful package for working with big datasets.
# Aside from providing helpful syntax, it is very efficient memory-wise.
# !pip install pandas
# !pip install os
# !pip install matplotlib


In [1]:
import pandas as pd
import os 

In [ ]:
# Set your working directory and load data

os.chdir("your_directory/")

hes = pd.read_csv("HES.csv")
ks1 = pd.read_csv("KS1.csv")
spine = pd.read_csv("spine.csv")

# Altenratively, you can download directly:
# hes <- read_csv("https://rdr.ucl.ac.uk/ndownloader/files/44972194")
# ks1 <- read_csv("https://rdr.ucl.ac.uk/ndownloader/files/44972182")
# spine <- read_csv("https://rdr.ucl.ac.uk/ndownloader/files/44972191")


In [4]:
hes.describe()

,epikey,epitype,epiorder,admimeth,startage,sex
count,5.000000e+05,500000.0,500000.0,500000.0,500000.0,500000.000000
mean,1.500980e+11,3.0,1.0,82.0,7001.0,1.500464
std,2.889684e+10,0.0,0.0,0.0,0.0,0.500000
min,1.000000e+11,3.0,1.0,82.0,7001.0,1.000000
25%,1.250595e+11,3.0,1.0,82.0,7001.0,1.000000
50%,1.501090e+11,3.0,1.0,82.0,7001.0,2.000000
75%,1.752093e+11,3.0,1.0,82.0,7001.0,2.000000
max,1.999999e+11,3.0,1.0,82.0,7001.0,2.000000


In [ ]:
# We are defining a birth cohort using HES data, so we'll focus there first.
# 1) What birth years have we provided you?
hes['epistart'] = pd.to_datetime(hes['epistart'])
hes['epistart_cyear'] = hes['epistart'].dt.year
birth_years = hes['epistart_cyear'].value_counts(dropna=False)
birth_years


In [ ]:
# 2) What is the start age for people in this dataset?
start_age = hes['startage'].value_counts(dropna=False)
start_age

In [ ]:
# 3) What epitype did we provide you? What does this mean?
epitype_counts = hes['epitype'].value_counts(dropna=False)
epitype_counts

In [ ]:
# 4) What admimeth did we provide you? What does this mean?
admimeth_counts = hes['admimeth'].value_counts(dropna=False)
admimeth_counts

In [ ]:
# 5) What ICD10 code did we provide you in DIAG_01? What does this mean?
diag_01_counts = hes['diag_01'].value_counts(dropna=False)
diag_01_counts

In [ ]:
# 6) How many unique individuals are there in the dataset?
unique_individuals = hes['tokenid'].nunique()
unique_individuals

In [ ]:
# 7) Identify which variable represents the region of residence.
hes.head(5)

In [ ]:
# 8) Calculate how many individuals' region of residence at birth is Scotland or Wales.
resgor_counts = hes['resgor'].value_counts(dropna=False)
scotland_wales_residence = hes[hes['resgor'].isin(['S', 'W'])]['resgor'].count()
scotland_wales_residence

In [ ]:
# 9) How many individuals are left once you only keep those who resided in English regions at birth?
english_residence_count = hes[~hes['resgor'].isin(['S', 'W'])]['resgor'].count
english_residence_count

In [ ]:
hes = hes[~hes['resgor'].isin(['S','W'])]

In [ ]:
### Part 2 ###
# 1) What ICD10 codes have we included in DIAG_02?

In [ ]:
diag_02_counts = hes['diag_02'].value_counts(dropna=False)
diag_02_counts

In [ ]:
# 2) Using the ICD-10 browser, what do these codes represent?

In [ ]:
# 3) How many patients have a major congenital anomaly code? Once you know, you can drop these patients
mca = ["Q24", "Q35", "Q37", "Q43"]
mca_count = hes[hes['diag_02'].isin(mca)].shape[0]
mca_proportion = mca_count / hes.shape[0]
mca_proportion

In [ ]:
# Now we can drop them
hes = hes[~hes['diag_02'].isin(mca)]

In [ ]:
# 4) How many patients have a linked record in the spine?
# Merging hes with spine
hes = pd.merge(hes, spine, on="tokenid", how="left")
no_link_count = hes['PupilMatchingRefAnonymous'].isna().sum()
link_count = len(hes) - no_link_count
link_count

In [ ]:
# Part 3
# 1) Are there differences in linkage rates between HES and NPD based on sex, deprivation and ethnicity?
# Once you have answered this question, you can drop the students without a link.


In [ ]:
# Let's first explore our sex, deprivation and ethnicity variables.
sex_counts = hes['sex'].value_counts(dropna=False)
ethnos_counts = hes['ethnos'].value_counts(dropna=False)
imd04_decile_counts = hes['imd04_decile'].value_counts(dropna=False)


In [ ]:
# Now examine non-linkage rates by each variable.
non_linkage_rates_sex = hes.groupby('sex')['PupilMatchingRefAnonymous'].apply(lambda x: f"{x.isna().sum()} ({round((x.isna().sum() / len(x)) * 100, 1)}%)")
non_linkage_rates_ethnos = hes.groupby('ethnos')['PupilMatchingRefAnonymous'].apply(lambda x: f"{x.isna().sum()} ({round((x.isna().sum() / len(x)) * 100, 1)}%)")
non_linkage_rates_imd04_decile = hes.groupby('imd04_decile')['PupilMatchingRefAnonymous'].apply(lambda x: f"{x.isna().sum()} ({round((x.isna().sum() / len(x)) * 100, 1)}%)")


In [ ]:
# Now we have analyzed linkage rates, We can drop those who did not link and focus on our research question.
hes = hes.dropna(subset=['PupilMatchingRefAnonymous'])

In [ ]:
# 2) What is the relationship between gestational age and KS1 results (unadjusted and adjusted for sex, deprivation and ethnicity)?
# Merging KS1 data with HES
hes_ks1_merge = pd.merge(hes, ks1[['PupilMatchingRefAnonymous', 'KS1_MATH']], on="PupilMatchingRefAnonymous", how="left")
len(hes_ks1_merge)

In [ ]:
# Plotting Histogram: KS1 Math
import matplotlib.pyplot as plt
plt.hist(hes_ks1_merge['KS1_MATH'], bins = 20, color = 'skyblue', edgecolor = 'black')
plt.xlabel('KS1 Math Scores')
plt.ylabel('Frequency')
plt.title('Histogram of HES linked with KS1 Math Scores')
plt.show()

In [ ]:
hes_ks1_merge['KS1_MATH'].describe()

In [ ]:
hes_ks1_merge['gestat'].value_counts()

In [ ]:
hes_ks1_merge['gestat_group'] = hes_ks1_merge['gestat'].astype('category')

# Reorder the categories with the reference level first
reference_level = '40'
hes_ks1_merge['gestat_group'] = hes_ks1_merge['gestat_group'].cat.reorder_categories([reference_level] + list(hes_ks1_merge['gestat_group'].cat.categories.difference([reference_level])))


In [ ]:
# Univariable analysis
univariable_analysis = hes_ks1_merge.groupby('gestat')['KS1_MATH'].agg(['mean', 'std'])
univariable_analysis

In [ ]:
hes_ks1_merge.dtypes


In [ ]:
import statsmodels.api as sm
import numpy as np

m1 = sm.OLS(hes_ks1_merge['KS1_MATH'], sm.add_constant(hes_ks1_merge['gestat'])).fit()
print(m1.summary())

In [ ]:
# m2: KS1_MATH ~ gestat + sex + imd04_decile + ethnos
m2 = sm.OLS(hes_ks1_merge['KS1_MATH'], sm.add_constant(hes_ks1_merge[['gestat_group', 'sex', 'imd04_decile', 'ethnos']])).fit()
print(m2.summary())